# Modeling Magnetic Saturation 

## Section 1:/ Introduction

### What is Ferromagnetic Saturation? 
Explain spunge/water analogy of ferro-magnetic material and magnetic flux. 
1. low magnitude of the magnetic field: ferromatic field absorbs the magnetic field (at low quantities of the water, the spunge absorbs the water); 

1. explain what saturation is and how it is modeled mathematically; 
2. choose example of $B$-$H$ curve; show linear behavior with small and large mur value; show how to derive $\mu_r$ and $d \mu_r / d \, normB$ from the curve (derivative of a cubic spline interpolation); 

### How is Ferromagnetic Saturation Modeled Mathematically? 
Absorption of magnetic field by ferro-magnetic material is governed by the magnetic permeability. High (low) magnetic permeability. Problem becomes non-linear. Numerical method to solve non-linear problems ought to be used.

### How to Numerically Solve the Resulting Non-Linear Problem? 
Using either a (damped) fixed point iteration (implemented e.g. by hand) or a Newton iteration (implemented in e.g. [NLSolve](https://github.com/JuliaNLSolvers/NLsolve.jl)). In the latter case, the Jacobian matrix requires evaluation. 


## Section 2:/ Three Parameter Model for B-H Curve with Saturation
[three-parameter-model](./three-parameter-model.ipynb)

## Section 3:/ Analytical Models for B-H Curve with Saturation 
[analytical-models-saturation](analytical-models-saturation.ipynb)

## Section 4:/ Jacobian Computation 
Assume mesh with $N$ elements and $N+1$ nodes. On on element e, the computed solution (approximation to the potential) is given by $u^h(x,y) = c_1 \, \phi_1(x,y) + c_2 \, \phi_2(x,y) + c_3 \, \phi_3(x,y)$. The magnetic flux vector is given by $B^h(x,y) = (c_1 \, b_1 + c_2 \, b_2 + c_3 \, b_3, - c_1 \, a_1 - c_2 \, a_2 - c_3 \, a_3)$. The norm of the magnetic flux square is given by $normB2 = (c_1 \, b_1 + c_2 \, b_2 + c_3 \, b_3)^2 + (c_1 \, a_1 + c_2 \, a_2 + c_3 \, a_3)^2$. From, this expression $\partial normB2 / \partial c_j$ for $1 \leq j \leq 3$ can be computed. 
$A_{ik} = \int_{\Omega} 1/\mu(normB2) \nabla \phi_i \cdot \nabla \phi_k$.  

$\begin{eqnarray} 
\partial A_{ik} / \partial c_j 
& = & \int_{\Omega} \partial  \left[ 1/\mu(normB2) \right] / \partial c_j 
      \nabla \phi_i \cdot \nabla \phi_k \\ 
& = & \int_{\Omega} \partial  \left[ 1/\mu(normB2) \right] / \partial normB2 \; 
      \partial normB2 / \partial c_j \; 
      \nabla \phi_i \cdot \nabla \phi_k
\end{eqnarray}$

Residual $\mathbf{R} \in R^{N+1}$ with components $R_i = \sum_{k=1}^{N} A_{ik} \, c_k - f_i$. 
Jacobian $J \in R^{(N+1) \times (N+1)}$ with components $J_{ij} = \partial R_i/\partial c_j = A_{ij} + \sum_{k \in \text{supp}(i)} \partial A_{ik} / \partial c_j \, c_k$. 

How to perform element-by-element assembly? 

#### Numerical part for 1D FDM on a uniform mesh: 

1. notation: we here assume 1D computations using the finite difference method on a uniform mesh. We will try to adhere to the notation in which the index $i$ numbers the nodes and thus $1 \leq i \leq N+1$, in which $j$ numbers the (midpoints of the) elements and thus $1 \leq j \leq N$ and in which the index $k$ is a summation index (to be made more precise). 

2. computation of normB: we assume normB to be constant per element. We therefore associate normB to the midpoints of the elements (instead of to the nodes). We provide here an expression for normB on midpoint of element as function of the potential in the nodes. The computation of normB requires the derivative of the computed potential. To compute the derivative of the computed potential on the element $[x_j, x_{j+1}]$, we first interpolate between the nodal values and subsequently differentiate the linear interpolant. We thus assume normB to be a function $\mathbf{normB}(\mathbf{c})$ from $\mathbf{c} \in R^{N+1}$ to $\mathbf{normB} \in R^{N}$; specify the components of this mapping $normB_j(\mathbf{c}) = \sqrt{c_j^2 + c_{j+1}^2/h$.   

3. fixed-point iteration of the non-linear solve: solve non-linear problem without Jacobian using fixed-point iteration; linear computation as initial guess; post-process for normB; retrieve mur for non-linear material characteristic tabulated as mur vs. Mur; repeat until convergence; 

4. expression of the Jacobian of the mapping $\mathbf{normB}(\mathbf{c})$ with elements 
$\partial_j \mathbf{normB}(\mathbf{c}) / \partial c_i $; 

5. solve non-linear problem with Jacobian using Newton iteration; provide function for non-linear residual and  its Jacobian to non-linear system solver such as solve() or find_zeros(); computation of the Jacobian will required more attention; provide details for the linear case first; in linear case R(\vec{c}) = A \vec{c} - f and J(\vec{c}) = A; in case of the 1D finite difference method on a uniform mesh A =B^T D B independent of \vec{c} where D_j is the value of the diffusion coefficient on the j-th element for $1 \leq j \leq N$ where $N$ is the number of elements; in non-linear case R(\vec{c}) = A(\vec{c}) \vec{c} - f and J(\vec{c}) = A + (dA/dc)*\vec{c}; in case of the 1D finite difference method on a uniform mesh dA/dc =B^T dD/dc B; in case of linear elements, $D_j$ depends on $c_j$ and $c_{j+1}$ via normB; on element $e_j = [x_j, x_{j+1}]$; normB is equal to $normB_j = \sqrt{c_j^2 + c_{j+1}^2/h_j$; therefore on $e_j = [x_j, x_{j+1}]$ $D_j = \mu_r(normB_j)$; therefore 

55. Jacobian dD/dc with elements dD/dc_{ij} for 1 <= i <= N and 1 <= j < = N+1. Jacobian is thus rectangular. The element dD/dc_{ij} is given by dD_i / dc_j  dD/dnormB dnormB/dc; dD/dnormB = dmur/dnormB from BH-curve interpolated using cubic splines;  need to make specific given de particular BH-curve; dnormB/dc from the expression of normB as function of c; 

5. look into packages such as zygote.jl and ChainRules; ask on discourse forum; maybe do finite differences first; provide references; 

#### Numerical part for 1D FEM on a possibly non-uniform mesh: 

1. notation: we here assume 1D computations with linear Lagrange elements. We will try to adhere to the notation in which the index $i$ numbers the nodes and thus $1 \leq i \leq N+1$, in which $j$ numbers the (midpoints of the) elements and thus $1 \leq j \leq N$ and in which the index $k$ is a summation index (to be made more precise). 

2. computation of normB: for linear element, normB is constant per element. We therefore associate normB to the midpoints of the elements (instead of to the nodes). expression for normB on midpoint of element as function of the potential in the nodes; function $\mathbf{normB}(\mathbf{c})$ from $\mathbf{c} \in R^{N+1}$ to $\mathbf{normB} \in R^{N}$; specify the components of this mapping $normB_j(\mathbf{c}) = \sqrt{c_j^2 + c_{j+1}^2/h_j$;

#### Use of Automatic Differentation  

1. discuss the use of automatic differentiation in Julia to compute the derive of scalar functions and the sparse Jacobian of vector valued function. Need to write assembly function such that automatic differentiation becomes feasible; 

#### Additions 
1. verify using gridap.jl, ferrite.jl or alternatives;